Thie notebook explores the multilayer perceptron for binary text classification, using the keras library.  Before starting, be sure to install the following versions of tensorflow and keras (for python 3.7):

```sh
pip install tensorflow==1.13.0-rc2
pip install keras==2.2.4
```

In [ ]:
import keras
import numpy as np
from sklearn import preprocessing
from keras.layers import Dense, Dropout
from keras.models import Sequential
from sklearn.feature_extraction.text import CountVectorizer
from keras.callbacks import ModelCheckpoint, EarlyStopping, Callback

In [ ]:
def read_data(filename):
    X=[]
    Y=[]
    with open(filename, encoding="utf-8") as file:
        for idx,line in enumerate(file):
            cols=line.rstrip().split("\t")
            label=cols[0]
            # assumes text is tokenized
            text=cols[1]
            X.append(text)
            Y.append(label)

    return X, Y

In [ ]:
directory="../data/lmrd"

In [ ]:
trainX, trainY=read_data("%s/train.tsv" % directory)
devX, devY=read_data("%s/dev.tsv" % directory)
testX, testY=read_data("%s/test.tsv" % directory)

vectorizer = CountVectorizer(max_features=10000, analyzer=str.split, lowercase=True, strip_accents=None, binary=True)
X_train = vectorizer.fit_transform(trainX)
X_dev = vectorizer.transform(devX)
X_test = vectorizer.transform(testX)

_,vocabSize=X_train.shape

le = preprocessing.LabelEncoder()
le.fit(trainY)

Y_train=le.transform(trainY)
Y_dev=le.transform(devY)
Y_test=le.transform(testY)

In [ ]:
def mlp():
    model = Sequential()
    model.add(Dense(10, activation='relu', input_shape=(vocabSize,)))
    model.add(Dropout(0.2))
    model.add(Dense(1, activation='sigmoid'))

    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['acc'])
        
    return model

In [ ]:
def train_and_evaluate(model):
    print (model.summary())

    # stop training early when the val_loss on the dev data stops going down
    early_stopping = EarlyStopping(monitor='val_loss',
        min_delta=0,
        patience=10,
        verbose=0, 
        mode='auto')

    # when training, save the model that has the best val_loss on the dev data
    modelName="mymodel.hdf5"
    checkpoint = ModelCheckpoint(modelName, monitor='val_loss', verbose=0, save_best_only=True, mode='min')

    # train
    model.fit(X_train, Y_train, 
                validation_data=(X_dev, Y_dev),
                epochs=5,
                callbacks=[checkpoint, early_stopping])
    
    # load best trained model (from checkpoint)
    model.load_weights(modelName)

    dev_loss, dev_accuracy = model.evaluate(X_dev, Y_dev, batch_size=128)
    print("Dev Accuracy: %.3f" % dev_accuracy)

    test_loss, test_accuracy = model.evaluate(X_test, Y_test, batch_size=128)
    print("Test Accuracy: %.3f" % test_accuracy)


In [ ]:
train_and_evaluate(mlp())